<a href="https://colab.research.google.com/github/DamnScallion/Visionary-Unicorns-Project/blob/main/yolo/train-dice-original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone files from our GitHub Repo

In [1]:
!git clone https://github.com/DamnScallion/Visionary-Unicorns-Project.git

Cloning into 'Visionary-Unicorns-Project'...
remote: Enumerating objects: 11038, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 11038 (delta 55), reused 20 (delta 11), pack-reused 10941
Receiving objects: 100% (11038/11038), 88.37 MiB | 13.26 MiB/s, done.
Resolving deltas: 100% (101/101), done.
Updating files: 100% (10840/10840), done.


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.0/605.0 kB 11.5 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os
import torch
import torch.nn as nn

In [4]:
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()

    def forward(self, pred, target):
        smooth = 1e-5
        intersection = (pred * target).sum()
        dice_coefficient = (2.0 * intersection + smooth) / (pred.sum() + target.sum() + smooth)
        dice_loss = 1.0 - dice_coefficient
        return dice_loss

In [5]:
class CustomBinaryDiceLoss(nn.Module):
    def __init__(self):
        super(CustomBinaryDiceLoss, self).__init__()
        self.dice_loss = DiceLoss()

    def forward(self, pred, target):
        # Assuming `pred` is the predicted output and `target` is the target tensor with binary labels
        loss = self.dice_loss(pred, target)
        return loss

In [6]:
class CustomYOLOv8n(YOLO):
    def __init__(self, config_path):
        super(CustomYOLOv8n, self).__init__(config_path)
        self.loss_fn = CustomBinaryDiceLoss()

    def forward(self, x):
        return self.model(x)

In [7]:
ROOT_DIR = '/content/Visionary-Unicorns-Project/yolo/config'

In [8]:
model = CustomYOLOv8n('yolov8n.yaml')


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [9]:
# Train the model with original given dataset for 100 epochs
results = model.train(data=os.path.join(ROOT_DIR, 'yolo_config.yaml'), epochs=100, imgsz=224, name='yolov8n_e10')

Ultralytics YOLOv8.0.143 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Visionary-Unicorns-Project/yolo/config/yolo_config.yaml, epochs=100, patience=50, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_e10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras

# Save the model to Google Drive

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [13]:
!scp -r runs/detect/yolov8n_e10 '/content/drive/MyDrive/COMP9517/YOLO_BCE result'